In [1]:
from datum import *
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.cluster import AffinityPropagation

%matplotlib inline 
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
#有中文出现的情况，需要u'内容'

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
data = Datum()
data.data_prepare()

In [28]:
weight_matrix = data.weight_matrix

In [29]:
weight_matrix.shape

(44, 3145, 2199)

In [30]:
weight_matrix_all = np.sum(weight_matrix[24:], axis=0)
year_index = np.array(range(25, 44, 2))
weight_matrix_year = np.sum(weight_matrix[year_index], axis=0)

weight_matrix[24:].shape

weight_matrix[year_index].shape

stable_stock_code = set()
for filename in os.listdir('data/select_stock_data'):
    code = filename.split('_')[0]
    for _ in range(6-len(code)):
        code = '0' + code     
    if code in stable_stock_code:
        continue
    else:
        stable_stock_code.add(code)

stable_index = []
for idx, stock in enumerate(data.list_stocks):
    if stock in stable_stock_code:
        stable_index.append(idx)
stable_index = np.array(stable_index)

weight_matrix_all = weight_matrix_all[stable_index, :]
weight_matrix_year = weight_matrix_year[stable_index, :]

weight_matrix_all.shape

weight_matrix_year.shape

list_stable_stocks = [data.list_stocks[i] for i in stable_index]

len(list_stable_stocks)

for weight_index, weight in enumerate([weight_matrix_all]):
    arr_tmp = []
    for stock_index in range(len(list_stable_stocks)):
        a_edge_index = np.where(weight[stock_index] != 0)[0]
        for edge_index in a_edge_index:
            arr_tmp.append([stock_index, len(list_stable_stocks) + edge_index, weight[stock_index, edge_index]])
    arr_tmp = np.array(arr_tmp)
    pd_tmp = pd.DataFrame(arr_tmp)
    pd_tmp[0] = pd_tmp[0].astype(int)
    pd_tmp[1] = pd_tmp[1].astype(int)
    pd_tmp.to_csv('data/graph/graph_stable_all.csv', index=False, sep=' ')

for weight_index, weight in enumerate([weight_matrix_year]):
    arr_tmp = []
    for stock_index in range(len(list_stable_stocks)):
        a_edge_index = np.where(weight[stock_index] != 0)[0]
        for edge_index in a_edge_index:
            arr_tmp.append([stock_index, len(list_stable_stocks) + edge_index, weight[stock_index, edge_index]])
    arr_tmp = np.array(arr_tmp)
    pd_tmp = pd.DataFrame(arr_tmp)
    pd_tmp[0] = pd_tmp[0].astype(int)
    pd_tmp[1] = pd_tmp[1].astype(int)
    pd_tmp.to_csv('data/graph/graph_stable_year.csv', index=False, sep=' ')

path = os.path.join('data/graph/graph_stable_all.csv')
nx_G = nx.read_edgelist(path, nodetype=int, data=(('weight', float),), create_using=nx.DiGraph())
nx_G = nx_G.to_undirected()
G = graph.Graph(nx_G, False, 1, 1)
G.preprocess_transition_probs()

walks = G.simulate_walks(50, 200)

walks = [list(map(str, walk)) for walk in walks]

model = Word2Vec(walks, size=32, window=6, min_count=0, sg=1, workers=2, iter=10)

model.wv.save_word2vec_format('data/embedding/stable_embedding_all.emb')

path = os.path.join('data/graph/graph_stable_year.csv')
nx_G = nx.read_edgelist(path, nodetype=int, data=(('weight', float),), create_using=nx.DiGraph())
nx_G = nx_G.to_undirected()
G = graph.Graph(nx_G, False, 1, 1)
G.preprocess_transition_probs()

walks = G.simulate_walks(50, 200)

walks = [list(map(str, walk)) for walk in walks]

model = Word2Vec(walks, size=32, window=6, min_count=0, sg=1, workers=2, iter=10)

model.wv.save_word2vec_format('data/embedding/stable_embedding_year.emb')

# embedding function

In [2]:
def embedding_cluster(filename):
    path = os.path.join('data/embedding/', filename+'.emb')
    emb_raw = pd.read_csv(path, header=None, sep=' ', skiprows=1)
    emb_raw = np.array(emb_raw)
    emb = np.zeros((emb_raw.shape[0], emb_raw.shape[1]-1))
    for idx, ele in enumerate(emb_raw):
        emb[idx] = ele[1:]  
    af = AffinityPropagation(preference=-50).fit(emb)
    distance = np.zeros((emb.shape[0], emb.shape[0]))
    for i in range(emb.shape[0]):
        for j in range(emb.shape[0]):
            distance[i][j] = np.linalg.norm(emb[i]-emb[j])
    f = open(filename+'.txt', 'w')
    cluster_num = 0
    for idx in af.cluster_centers_indices_:
        f.write('new cluster\n')
        cluster_idx = np.where(af.labels_ == af.labels_[idx])[0]
        sli = distance[idx]
        arg_rank = np.argsort(sli[cluster_idx])
        for arg in arg_rank:
            arg_true = cluster_idx[arg]
            if arg_true >= len(data.use_index):
                continue
            code = data.list_stocks[use_index][arg_true]       
            try:
                name = data.dict_code2name[code].split(';')[-2]
            except:
                name = 'Not know'
            f.write(str(cluster_num)+','+str(arg_true)+', '+code+', '+name+'\n')
        cluster_num = cluster_num + 1
        f.write('\n\n\n\n')

In [3]:
def get_data(start_date, end_date, stock_code):
    price_csv = pd.read_csv('data/stock_data/'+str(stock_code)+'_price.csv')
    dates = np.array(price_csv['TradingDay'])
    for count in range(len(dates)):
        dates[count] = int(''.join(str(dates[count]).split()[0].split('-')))
    select_index = np.where((dates >= start_date) & (dates < end_date))[0]
    return_price = np.array(price_csv['ClosePrice'][select_index])
    
    feature_csv = pd.read_csv('data/stock_data/'+str(stock_code)+'_feature.csv')
    dates = np.array(feature_csv['trading_day'])
    for count in range(len(dates)):
        dates[count] = int(''.join(str(dates[count]).split()[0].split('-')))
    select_index = np.where((dates >= start_date) & (dates < end_date))[0]
    return_feature = np.array(feature_csv)[select_index, 1:]  
    
    return return_price, return_feature

num_set = set()
for filename in os.listdir('data/stock_data/'):
    num = filename.split('_')[0]
    if num in num_set:
        continue
    else:
        num_set.add(num) 
data = Datum()
data.data_prepare()  
# stock select: 2016,2015...
# season_select: all, year
# emb_start:2012, 2013...
def embedding(emb_start='None', stock_select='None', season_select='all'):
    if stock_select != 'None':
        stock_select = int(stock_select)
        day_feature = get_data(stock_select*10000, 20180000, 1)[1].shape[0]
        day_price = get_data(stock_select*10000, 20180000, 1)[0].shape[0]
        select_num_set = set()
        for num in num_set:
            price, feature = get_data(stock_select*10000, 20180000, int(num))
            if price.shape[0] == day_price and feature.shape[0] == day_feature:
                select_num_set.add(num)   
    else:
        stock_select = 'all'
        select_num_set = num_set
    
   
    
    stable_stock_code = set()
    for num in select_num_set:
        code = num
        for _ in range(6-len(code)):
            code = '0' + code     
        if code in stable_stock_code:
            continue
        else:
            stable_stock_code.add(code)    
    
    stable_index = []
    for idx, stock in enumerate(data.list_stocks):
        if stock in stable_stock_code:
            stable_index.append(idx)
    stable_index = np.array(stable_index)    
    data.use_index = stable_index
    list_stable_stocks = [data.list_stocks[i] for i in stable_index]   
    
    weight_matrix = np.copy(data.weight_matrix)
    if emb_start != 'None':
        emb_start = int(emb_start)
        if season_select == 'all':
            weight_matrix_total = np.sum(weight_matrix[list(data.select_date).index(emb_start*10000):], axis=0)
            print(weight_matrix[list(data.select_date).index(emb_start*10000):].shape)
        else:
            index = np.array(range(list(data.select_date).index(emb_start*10000)+1, 44, 2))
            weight_matrix_total = np.sum(weight_matrix[index], axis=0)
            print(weight_matrix[index].shape)
    else:
        emb_start = 'all'
        if season_select == 'all':
            weight_matrix_total = np.sum(weight_matrix, axis=0)
            print(weight_matrix.shape)
        else:
            index = np.array(range(1, 44, 2))
            weight_matrix_total = np.sum(weight_matrix[index], axis=0)
            print(weight_matrix[index].shape)        

    weight_matrix_total = weight_matrix_total[stable_index, :]

    for weight_index, weight in enumerate([weight_matrix_total]):
        arr_tmp = []
        for stock_index in range(len(list_stable_stocks)):
            a_edge_index = np.where(weight[stock_index] != 0)[0]
            for edge_index in a_edge_index:
                arr_tmp.append([stock_index, len(list_stable_stocks) + edge_index, weight[stock_index, edge_index]])
        arr_tmp = np.array(arr_tmp)
        pd_tmp = pd.DataFrame(arr_tmp)
        pd_tmp[0] = pd_tmp[0].astype(int)
        pd_tmp[1] = pd_tmp[1].astype(int)
        pd_tmp.to_csv('data/graph/graph_{}_{}_{}.csv'.format(emb_start, stock_select, season_select), index=False, sep=' ')


        path = 'data/graph/graph_{}_{}_{}.csv'.format(emb_start, stock_select, season_select)
        nx_G = nx.read_edgelist(path, nodetype=int, data=(('weight', float),), create_using=nx.DiGraph())
        nx_G = nx_G.to_undirected()
        G = graph.Graph(nx_G, False, 1, 1)
        G.preprocess_transition_probs()

        walks = G.simulate_walks(50, 200)
        walks = [list(map(str, walk)) for walk in walks]
        model = Word2Vec(walks, size=32, window=6, min_count=0, sg=1, workers=2, iter=10)
        model.wv.save_word2vec_format('data/embedding/embedding_{}_{}_{}.emb'.format(emb_start, stock_select, season_select))
        
    embedding_cluster('embedding_{}_{}_{}.emb'.format(emb_start, stock_select, season_select))


In [ ]:
embedding(2016, 0, 'all')

(4, 3145, 2199)
Walk iteration:
('1', '/', '50')
('2', '/', '50')
('3', '/', '50')
('4', '/', '50')
('5', '/', '50')
('6', '/', '50')
('7', '/', '50')
('8', '/', '50')
('9', '/', '50')
('10', '/', '50')
('11', '/', '50')
('12', '/', '50')
('13', '/', '50')
('14', '/', '50')
('15', '/', '50')
('16', '/', '50')
('17', '/', '50')
('18', '/', '50')
('19', '/', '50')
('20', '/', '50')
('21', '/', '50')
('22', '/', '50')
('23', '/', '50')
('24', '/', '50')
('25', '/', '50')
('26', '/', '50')
('27', '/', '50')
('28', '/', '50')
('29', '/', '50')
('30', '/', '50')
('31', '/', '50')
('32', '/', '50')
('33', '/', '50')
('34', '/', '50')
('35', '/', '50')
('36', '/', '50')
('37', '/', '50')
('38', '/', '50')
('39', '/', '50')
('40', '/', '50')
('41', '/', '50')
('42', '/', '50')
('43', '/', '50')
('44', '/', '50')
('45', '/', '50')
('46', '/', '50')
('47', '/', '50')
('48', '/', '50')
('49', '/', '50')
('50', '/', '50')
